In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import rioxarray
import xesmf as xe

from cdo import Cdo

cdo = Cdo()

# https://figshare.com/articles/dataset/CROPGRIDS/22491997

# Regrid cropland masks

In [ ]:
GCM = "mpi"
# GCM = "cesm2"

for crop in ("rice", "soybean", "maize", "wheat"):

    if crop == "wheat":
        crop_name = "wht"
    elif crop == "maize":
        crop_name = "gro"
    elif crop == "rice":
        crop_name = "pdr"
    elif crop == "soybean":
        crop_name = "osd"
    else:
        raise NotImplementedError()

    crop_da_path = "data/cropgrid/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_" + crop + ".nc"

    # Open into an xarray.DataArray
    crop_da = xr.open_dataset(crop_da_path)["harvarea"]
    crop_da = crop_da.rename("data")

    data_lons = crop_da["lon"].values
    i = np.where(data_lons <= 0)
    data_lons[i] = data_lons[i] + 360
    i = np.where(data_lons >= 359.93)
    data_lons[i] = 0.0
    crop_da = crop_da.assign_coords(lon=data_lons)
    crop_da = crop_da.sortby("lon")

    # convert to harvested area
    total_area = xr.open_dataarray("data/cropgrid/gridarea.nc")
    crop_da = (crop_da * 10000.0) / total_area.values

    crop_da = xr.where(crop_da < 0, 0.0, crop_da)
    crop_da = xr.where(crop_da > 1.0, 1.0, crop_da)

    crop_da = crop_da.assign_coords(
        {
            "lon": (["lon"], crop_da["lon"].values, {"units": "degrees_east"}),
            "lat": (["lat"], crop_da["lat"].values, {"units": "degrees_north"}),
        }
    )

    # get the data grid
    if GCM == "mpi":
        data = xr.open_dataarray("processed_data/exp501_processed_data.nc")
    elif GCM == "cesm2":
        data = xr.open_dataarray("processed_data/exp000_processed_data.nc")

    data = data.stack(sample=("member", "window")).transpose("sample", "lat", "lon")
    data = data[0, :, :]
    data = data.drop_vars(("member", "window", "sample"))
    data.to_netcdf("data/cropgrid/regridded_nc/" + GCM + "_basegrid.nc")

    # regrid the data
    output_filename = "data/cropgrid/regridded_nc/" + crop_name + "_harv_areafrac_" + GCM + "_regrid.nc"
    cdo.remapcon(
        "data/cropgrid/regridded_nc/" + GCM + "_basegrid.nc",
        input=crop_da,
        output=output_filename,
    )
    crop_da_regrid = xr.open_dataarray(output_filename)

In [ ]:
raise ValueError

## Regrid Irrigation Data

In [ ]:
for GCM in ("mpi", "cesm2"):

    geotiff_da = rioxarray.open_rasterio("data/irrigation/g_aei_2015.tif")
    irr_da = geotiff_da.to_dataset("band")
    irr_da = irr_da.rename({1: "data"})["data"]
    irr_da = irr_da.rename({"x": "lon", "y": "lat"})

    data_lons = irr_da["lon"].values
    i = np.where(data_lons <= 0)
    data_lons[i] = data_lons[i] + 360
    i = np.where(data_lons >= 359.93)
    data_lons[i] = 0.0
    irr_da = irr_da.assign_coords(lon=data_lons)
    irr_da = irr_da.sortby("lon")

    irr_da = xr.where(irr_da < 0, 0.0, irr_da)
    irr_da = irr_da.drop_vars("spatial_ref")
    irr_da = irr_da.assign_coords(
        {
            "lon": (["lon"], irr_da["lon"].values, {"units": "degrees_east"}),
            "lat": (["lat"], irr_da["lat"].values, {"units": "degrees_north"}),
        }
    )
    # save for making areagrid with cdo
    irr_da.to_netcdf("data/irrigation/g_aei_2015.nc")
    #>> cdo gridarea g_aei_2015.nc gridarea.nc

    # convert to harvested area
    total_area = xr.open_dataarray("data/irrigation/gridarea.nc")
    irr_da = (irr_da * 10000.0) / total_area.values

    irr_da = xr.where(irr_da < 0, 0.0, irr_da)
    irr_da = xr.where(irr_da > 1.0, 1.0, irr_da)

    irr_da = irr_da.assign_coords(
        {
            "lon": (["lon"], irr_da["lon"].values, {"units": "degrees_east"}),
            "lat": (["lat"], irr_da["lat"].values, {"units": "degrees_north"}),
        }
    )

    # regrid
    # get the data grid
    if GCM == "mpi":
        data = xr.open_dataarray("processed_data/exp501_processed_data.nc")
    elif GCM == "cesm2":
        data = xr.open_dataarray("processed_data/exp000_processed_data.nc")

    data = data.stack(sample=("member", "window")).transpose("sample", "lat", "lon")
    data = data[0, :, :]
    data = data.drop_vars(("member", "window", "sample"))
    data.to_netcdf("data/irrigation/" + GCM + "_basegrid.nc")

    # regrid the data
    output_filename = "data/irrigation/" + "irrigation_" + GCM + "_regrid.nc"
    cdo.remapcon(
        "data/irrigation/" + GCM + "_basegrid.nc",
        input=irr_da,
        output=output_filename,
    )
    irr_da_regrid = xr.open_dataarray(output_filename)